In [68]:
import os, sys, email, datetime, pprint, re, time, html, pickle
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *

pd.options.display.max_colwidth = 1000

In [69]:
### 自然言語処理
import spacy 
nlp = spacy.load("en_core_web_sm")

from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
nlp.add_pipe(sentencizer)

In [70]:
with open('/Users/taroaso/myprojects/OpenIE/trec/output/tagme.pickle', mode="rb") as f:
    tagme_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/ner.pickle', mode="rb") as f:
    ner_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/type_translation.pickle', mode="rb") as f:
    translation = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/openie_result.pickle', mode="rb") as f:
    openie = pickle.load(f)

In [42]:
from db import connect
engine = connect()
mail_df = pd.read_sql(sql='SELECT DISTINCT id, body FROM mail',con=engine, index_col=None)

In [56]:
doc = nlp(mail_df[mail_df['id']=='4084867E.9050502@kbc.net.au']['body'].values[0])

TypeError: to_array() takes exactly one argument (0 given)

In [67]:
test = {}
for sent in doc.sents:
    print((sent.text,sent.start,sent.end))

('Jesper Tverskov writes:\n One of the recommendations from City University on the WAI Guidelines said:\n - improve search design \n<content trimmed/>\n ', 0, 29)
("Now, from the point of view of accessibility, when a website's pages are indexed by Google, wouldn't it be easier for people with disabilities to use a seach engine they are already used to use?", 29, 70)
('Should we actively promote the use of search engines like Google as local seach engine at any major website, if the pages are already indexed by Google?', 70, 99)
('\n', 99, 100)
('Matthew Smith replies:\n', 100, 105)
('Whilst I think that Google has a great product, I do not think it in the best \ninterests of accessibility, or indeed of a stable infrastructure, to standardise \non a single, proprietary solution.', 105, 145)
('\n', 145, 146)
('If Google should fail as a company or start charging excessively for the Site \nSearch service), if everyone were using it, the entire concept of Site Search \nwould drop into a v

In [53]:
openie['4084867E.9050502@kbc.net.au']

[{'facts': [{'subject': 'website', 'predicate': 'has', 'object': 'pages'},
   {'subject': "website 's pages",
    'predicate': 'are indexed by',
    'object': 'Google'},
   {'subject': 'it', 'predicate': 'be', 'object': 'easier'},
   {'subject': 'it',
    'predicate': 'be easier from',
    'object': 'point of view of accessibility'},
   {'subject': 'it',
    'predicate': 'be easier for',
    'object': 'people with disabilities to use seach engine'},
   {'subject': 'disabilities',
    'predicate': 'be use',
    'object': 'seach engine'},
   {'subject': 'they',
    'predicate': 'are used to use',
    'object': 'seach engine already'},
   {'subject': 'they',
    'predicate': 'are used to use',
    'object': 'seach engine'}]},
 {'facts': [{'subject': 'Google',
    'predicate': 'is',
    'object': 'search engine'},
   {'subject': 'we', 'predicate': 'promote use of', 'object': 'Google'},
   {'subject': 'we',
    'predicate': 'promote use of Google as local seach engine at',
    'object': 'QU

In [38]:
oie_triples = []
for uid in openie.keys():
    # トリプルの抽出結果とエンティティの抽出結果があるときのみ,処理を継続する
    if openie[uid] != [] and tagme_result[uid] is not None:
        # メールごとにopenIEで抽出された複数のトリプルを取得する
        for oie in openie[uid]:
            # トリプルを走査する
            for fact in oie['facts']:
                # tagmeのアノテーション結果を走査する
                for annotation in tagme_result[uid]['annotations']:         
                    # トリプルの主語と一致するspotがある場合，トリプル（主語，メンション，検出されたスポットのリソース）作成
                    if annotation['spot'] == fact['subject'] and annotation['link_probability'] >= 0.3: 
                        emailmessage = uid
                        #emailmessage = re.sub(r'[^a-zA-Z_0-9]','_',uid)
                        start = annotation['start']
                        end = annotation['end']
                        offset_based_string = emailmessage + '#offset_' + str(start) + '_' + str(end)
                        oie_triples.append((fact['subject'],'mentions',offset_based_string))
                        #　トリプルの目的語と一致するspotがある場合，（目的語，メンション，検出されたスポットのリソース）作成
                        for annotation in tagme_result[uid]['annotations']: 
                            if annotation['spot'] == fact['object'] and annotation['link_probability'] >= 0.3:
                                start = annotation['start']
                                end = annotation['end']
                                offset_based_string = emailmessage + '#offset_' + str(start) + '_' + str(end)
                                test.append((fact['object'],'mentions',offset_based_string))
                            else:
                                pass
                        #openIEのトリプル作成
                        oie_triples.append((fact['subject'],fact['predicate'],fact['object']))
                    else:
                        pass

In [39]:
len(oie_triples)

1320